In [134]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics 
import json
import ast
from utils import util
import seaborn as sns

In [130]:
df_rotamer_pkl = pd.read_pickle('df_rotamer.pkl')

In [131]:
df_rotamer_pkl = df_rotamer_pkl.explode(['bound_residue'])
df_rotamer_pkl.dropna(inplace=True)
#df_rotamer_pkl['encoded_aa']=encoded_residue

In [132]:
x = df_rotamer_pkl.drop(columns=["pdb","metal","ion","motif_binding_xyz","num_NN_atom","NN_atom_properties",'paper_bound_residue','pdb_id'])
x = pd.get_dummies(x,drop_first=True)
y = df_rotamer_pkl["metal"]
y = pd.get_dummies(y,drop_first=True)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.1)

In [135]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train,y_train)
y_pred = dtc.predict(x_test)
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.5827814569536424


In [140]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.5933774834437087


In [143]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
y = df_rotamer_pkl["metal"]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.1)
gbc.fit(x_train,y_train)
y_pred = gbc.predict(x_test)
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.6052980132450331
